In [1]:
from bertopic import BERTopic
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jensenshannon
import pandas as pd
import numpy as np
import json

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/hgu/opt/anaconda3/envs/openai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('dim_sample.json') as f:
    all_data = json.load(f)

def get_unique_dis(data):
    disciplines = set()
    for pro_id in data.keys():
        disciplines.update(data[pro_id]['disciplines'])
    return disciplines

In [3]:
split = 180
train_data = {k: all_data[k] for k in list(all_data.keys())[:split]}
test_data = {k: all_data[k] for k in list(all_data.keys())[split:]}

train_project_data = {}
train_pub_data = []
for pro_id in train_data.keys():
    train_project_data[pro_id] = (train_data[pro_id]['title'], train_data[pro_id]['disciplines'])
    for pub_id in train_data[pro_id]['publications'].keys():
        train_pub_data.append((pub_id, train_data[pro_id]['publications'][pub_id]['title'], train_data[pro_id]['publications'][pub_id]['disciplines']))

test_project_data = {}
test_pub_data = []
for pro_id in test_data.keys():
    test_project_data[pro_id] = (test_data[pro_id]['title'], test_data[pro_id]['disciplines'])
    for pub_id in test_data[pro_id]['publications'].keys():
        test_pub_data.append((pub_id, test_data[pro_id]['publications'][pub_id]['title'], test_data[pro_id]['publications'][pub_id]['disciplines']))

print(len(train_project_data), len(train_pub_data))
print(len(test_project_data), len(test_pub_data))
print(len(get_unique_dis(train_data)), len(get_unique_dis({pub[0]: {'disciplines': pub[2]} for pub in train_pub_data})))

180 506
45 129
170 170


In [4]:
train_docs = [project[0] for project in train_project_data.values()] + [pub[1] for pub in train_pub_data]
test_docs = [project[0] for project in test_project_data.values()] + [pub[1] for pub in test_pub_data]

In [5]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(train_docs)

2024-06-16 19:28:51,630 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 22/22 [00:03<00:00,  6.68it/s]
2024-06-16 19:28:57,327 - BERTopic - Embedding - Completed ✓
2024-06-16 19:28:57,327 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-06-16 19:29:02,407 - BERTopic - Dimensionality - Completed ✓
2024-06-16 19:29:02,408 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-16 19:29:02,446 - BERTopic - Cluster - Completed ✓
2024-06-16 19:29:02,449 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-16 19:29:02,475 - BERTopic - Representation - Completed ✓


In [7]:
project_abstracts = [project[0] for project in train_project_data.values()] + [project[0] for project in test_project_data.values()]
pub_abstracts = [pub[1] for pub in train_pub_data] + [pub[1] for pub in test_pub_data]

project_matrix = topic_model.approximate_distribution(project_abstracts)[0]

pub_matrix = topic_model.approximate_distribution(pub_abstracts)[0]

100%|██████████| 1/1 [00:00<00:00, 26.97it/s]


In [8]:
distance_matrix = np.zeros((len(project_matrix), len(pub_matrix)))

for i in range(len(project_matrix)):
    for j in range(len(pub_matrix)):
        distance_matrix[i, j] = 1 - cosine_similarity([project_matrix[i]], [pub_matrix[j]])[0][0]

distance_matrix.shape

(225, 635)

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# create a dataframe with row indices as project ids and column indices as publication ids
# if publication is in project, then 1, else 0
# if publication date is before project start date, then 0, else 1
# ratio of intersection of organisations from project and publication to organisations from publication
# ratio of intersection of authors from project and publication to authors from publication

all_data = {**train_data, **test_data}
pub_data = []
for pro_id in all_data.keys():
    for pub_id in all_data[pro_id]['publications'].keys():
        pub_data.append((pub_id, all_data[pro_id]['publications'][pub_id]))

data_matrix = [[0.] * len(pub_data) for _ in range(len(all_data))]
for i, pro_id in enumerate(all_data.keys()):
    for j, pub_id in enumerate([pub[0] for pub in pub_data]):
        belongs = 1 if pub_id in all_data[pro_id]['publications'].keys() else 0
        distance = distance_matrix[i, j]
        data_matrix[i][j] = [belongs, distance]

df = pd.DataFrame(distance_matrix, index=all_data.keys(), columns=[pub[0] for pub in pub_data])
df.to_csv('distance_bert_dim_title.csv')
df.head()

,10.3389/fmars.2023.1254461,10.1007/978-3-031-49133-7_8,10.1007/978-3-031-43619-2_15,10.1080/11663081.2023.2246863,10.1177/03057356231165327,10.1016/j.jconrel.2024.05.037,10.1007/s13346-024-01618-6,10.1002/smll.202303682,10.1371/journal.pone.0298895,10.1007/s00334-024-00992-y,10.1038/s41598-023-44445-4,10.1101/2023.09.15.557939,10.1016/j.jasrep.2023.104204,10.1016/j.palaeo.2023.111655,10.1007/jhep05(2024)154,10.1140/epjc/s10052-024-12711-y,10.1140/epjc/s10052-024-12416-2,10.1140/epjc/s10052-024-12415-3,10.1007/jhep02(2024)124,10.1007/jhep11(2023)103,10.1007/jhep06(2023)086,10.1093/iob/obad026,10.1007/s00359-023-01629-7,10.1101/2023.03.11.531676,10.1093/mnras/stae997,10.1093/mnras/stae616,10.1051/0004-6361/202348958,10.1093/mnras/stad3163,10.1093/mnras/stad2183,10.1101/2024.01.14.574481,10.1007/s40313-024-01095-9,10.1109/tpwrs.2023.3321700,10.1109/tits.2024.3392914,10.1007/978-981-97-0989-2_24,10.1109/tsc.2023.3332701,10.1007/978-3-031-48424-7_15,10.1007/978-3-031-48424-7_13,10.1145/3631353,10.1145/3629517,10.1109/icdcs57875.2023.00111,10.1109/icws60048.2023.00056,10.1109/icws60048.2023.00031,10.1109/icws60048.2023.00068,10.1109/tsc.2023.3307143,10.1145/3600232,10.1109/mic.2023.3267266,10.1109/tsc.2023.3241975,10.1109/tmc.2022.3230856,10.1007/978-3-031-20984-0_9,10.1109/tsc.2022.3160469,10.3389/fneur.2023.1182561,10.1038/s41598-024-57498-w,10.1016/j.jmrt.2023.11.014,10.1016/j.jmst.2023.05.020,10.1016/j.fuel.2023.130187,10.1017/9781009297349,10.1007/s11229-023-04282-4,10.3390/rel14060758,10.1002/smsc.202400096,10.1016/j.actpsy.2024.104252,10.1080/02691728.2024.2326828,10.5964/ejop.12031,10.1111/jopy.12805,10.1109/lsp.2023.3277792,10.21203/rs.3.rs-3644422/v1,10.1016/j.renene.2024.120101,10.1093/rheumatology/kead301,10.1097/mlr.0000000000002000,10.1111/tmi.13827,10.1063/5.0167167,10.1021/acsami.2c19376,10.1021/acsanm.2c05047,10.1101/2024.05.21.595254,10.1038/s41467-024-47964-4,10.1093/jb/mvae021,10.1016/j.neuron.2024.01.014,10.1016/j.isci.2023.107385,10.1016/j.peptides.2023.171051,10.3389/fphar.2023.1138666,10.3389/fphar.2023.1098976,10.1186/s43058-023-00521-4,10.1177/26334895231199467,10.1002/adom.202303199,10.1002/adom.202301232,10.1016/j.energy.2024.130322,10.1126/sciimmunol.abo4365,10.1515/sem-2023-0178,10.1364/boe.506664,10.1364/boe.494585,10.3847/1538-4357/ac4601,10.1371/journal.pone.0284416,10.1080/07350015.2023.2271039,10.1093/mnras/stae385,10.1093/mnras/stad2760,10.1002/ange.202320045,10.1002/anie.202320045,10.1016/j.chembiol.2024.02.012,10.1101/2023.11.29.569293,10.1038/s41467-023-41197-7,10.35772/ghm.2023.01065,10.3390/v15040896,10.1111/dar.13836,10.1063/5.0153862,10.1109/imoc57131.2023.10379741,10.1109/imoc57131.2023.10379729,10.1109/access.2023.3339563,10.1109/ojpel.2023.3339014,10.1109/csci58124.2022.00203,10.1016/j.ssci.2022.105967,10.1007/978-3-031-06015-1_8,10.1101/2023.01.19.524726,10.1007/s00382-023-07058-1,10.1002/oa.3284,10.1136/bmjopen-2023-080603,10.1101/2023.08.02.551649,10.1099/mgen.0.000965,10.2140/ant.2022.16.2005,10.1103/physrevlett.132.103402,10.21468/scipostphys.15.2.068,10.1080/02606755.2023.2213550,10.3390/rs15204928,10.1063/5.0203603,10.1103/physrevapplied.21.054027,10.1109/icecom58258.2023.10367924,10.1109/icecom58258.2023.10367934,10.1109/icecom58258.2023.10367938,10.1109/icecom58258.2023.10367923,10.1109/metamaterials58257.2023.10289546,10.23919/ursigass57860.2023.10265525,10.1109/elmar59410.2023.10253914,10.1002/pen.26771,10.1016/j.canep.2023.102451,10.3390/biomedicines11051344,10.3390/pharmaceutics15010181,10.1037/fam0001189,10.3390/su15129371,10.1038/s44221-024-00208-7,10.1093/evolut/qpae036,10.1093/evolut/qpae024,10.3390/micro4010008,10.3390/molecules28196822,10.1007/s11673-023-10285-0,10.1021/acsnano.3c09776,10.1101/2023.09.08.556607,10.1016/j.snb.2023.133733,10.1016/j.ocecoaman.2023.106961,10.1016/j.tre.2023.103169,10.1016/j.elerap.2023.101336,10.1590/0001-3765202320230351,10.1590/s1678-3921.pab2023.v58.03246,10.1038/s41598-024-56540-1,10.1364/ol.494369,10.1017/s0963180124000112,10.1136/jm

In [12]:
data_matrix = np.array(data_matrix)

gb_raw_shape = data_matrix.shape
print(gb_raw_shape)

gb_data = []
for i in range(gb_raw_shape[1]):
    gb_data.extend(data_matrix[:, i, :])
gb_data = np.array(gb_data)
print(gb_data.shape)

project_numbers = np.array(list(range(1, gb_raw_shape[0] + 1)) * gb_raw_shape[1])

project_numbers = np.array(list(range(1, gb_raw_shape[0] + 1)) * gb_raw_shape[1])

# get the projects for lowest predictions for each publication with distance matrix
for num_top in range(10, 0, -1):
    project_numbers_pred = []
    for i in range(len(train_pub_data), distance_matrix.shape[1]):
        project_numbers_pred.append(list(np.argsort(distance_matrix[:, i] + 1))[:num_top])

    project_lists = []
    for i in range(0, gb_data.shape[0], gb_raw_shape[0]):
        pub_belongs = gb_data[i:i + gb_raw_shape[0], 0] * project_numbers[i:i + gb_raw_shape[0]]
        project = sorted(list(pub_belongs[pub_belongs != 0].astype(int)), reverse=True)
        project_lists.append(project)

    project_lists_test = project_lists[len(train_pub_data):]

    # calculate accuracy
    # use y_test as mask for project_numbers_test
    accuracy_list = []
    for i, project_list in enumerate(list(project_lists_test)):
        ratio = len(set(project_numbers_pred[i]) & set(project_list)) / len(set(project_list))
        accuracy_list.append(ratio)

    accuracy = sum(accuracy_list) / len(accuracy_list)
    print(accuracy)

(225, 635, 2)
(142875, 2)
0.03875968992248062
0.03875968992248062
0.03875968992248062
0.031007751937984496
0.023255813953488372
0.023255813953488372
0.023255813953488372
0.015503875968992248
0.007751937984496124
0.007751937984496124
